# Le Concept de Messages

Les *messages* représentent des échanges structurés dans une conversation entre un utilisateur, un agent IA, ou des outils externes. C’est une abstraction clé pour travailler efficacement avec des modèles de type **Grands Modèles de Languages**.

---

## 1. Pourquoi structurer les messages ?

Structurer les échanges en objets `Message` permet de :

- ✅ Maintenir l’historique contextuel de manière organisée.
- ✅ Définir le rôle de chaque message (utilisateur, IA, système, etc.).
- ✅ Faciliter la gestion de la mémoire conversationnelle.
- ✅ Permettre l’intégration avec des outils ou des fonctions externes.
- ✅ Créer des **agents intelligents** capables d'interagir, réfléchir et agir étape par étape.

---

## 2. Les types de messages

| Type de Message     | Description                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| `SystemMessage`     | Instructions initiales données à l’IA (définition du rôle, ton, etc.)       |
| `HumanMessage`      | Message envoyé par l’utilisateur humain.                                    |
| `AIMessage`         | Réponse générée par le modèle IA.                                           |
| `FunctionMessage`   | Résultat d’un appel de fonction exécuté par le modèle.                      |
| `ToolMessage`       | Message provenant d’un outil utilisé dans le cadre d’un agent.              |

Chaque message peut contenir du **contenu** (texte, fonction, résultats), des **métadonnées**, et participer à une session enrichie et pilotable.


In [4]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.chat_models import init_chat_model

messages = [
    SystemMessage(content="tu est un assistant"),
    HumanMessage(content="fais 5 ffois 5"),
    # AIMessage(content="Très bien. Souhaitez-vous un hôtel ou une activité spécifique ?")
]

llm = init_chat_model(model = "gpt-4o-mini" , model_provider="openai")
call = llm.invoke(messages)
print(call)

content='5 fois 5 égale 25.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 23, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BXppgYBjPepodyBxYB58Nr2pFhWgb', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--d78fa472-9416-4b5a-b6fa-03eb8859e901-0' usage_metadata={'input_tokens': 23, 'output_tokens': 10, 'total_tokens': 33, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### Personnalisation du comportement du modèle

Une **hallucination** dans le contexte de l’intelligence artificielle désigne une réponse générée par le modèle qui semble plausible, mais qui est **fausse, inventée ou incohérente avec les données réelles**.

Cela peut se produire lorsque le modèle :

- ne dispose pas d’un contexte suffisant,
- interprète mal une consigne,
- ou tente de compléter une réponse sans disposer d'informations fiables.

Un prompt bien conçu permet de :

- **fournir un cadre clair** (ex. : "réponds uniquement si tu connais la réponse"),
- **spécifier la source d’information** (ex. : "utilise uniquement les données ci-dessous"),
- **imposer une structure** de réponse (tableau, JSON, résumé...),
- **restreindre le domaine** de la réponse (ex. : "ne parle que de la législation congolaise").

#### ✅ Exemple

Prompt flou :
> *"Explique la législation sur les start-ups."*

➡️ Réponse possible : mélange de lois de différents pays, imprécisions.

Prompt guidé :
> *"Tu es un juriste congolais. Explique uniquement les lois en vigueur en République du Congo concernant la création de start-ups, en t’appuyant sur l’Acte uniforme OHADA."*


### String PromptTemplates

Ces modèles de prompts sont utilisés pour formater une seule chaîne de caractères et sont généralement utilisés pour des entrées plus simples.

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tu es un traducteur. Tu traduit le {contexte} en {langue}")

prompt_template.invoke({"contexte": "Bonjour", "langue" : "anglais"})

StringPromptValue(text='Tu es un traducteur. Tu traduit le Bonjour en anglais')

### ChatPromptTemplates

Ces modèles d'invite sont utilisés pour formater une liste de messages. Ces « modèles » consistent en une liste de modèles eux-mêmes. Par exemple, une façon courante de construire et d'utiliser un ChatPromptTemplate est la suivante

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "Tu es un Rose, une slameuse"),
    ("user", "Ecrit un texte de slam sur le {sujet}")
])

prompt_template.invoke({"sujet": "amour"})

#Dans l'exemple ci-dessus, ce ChatPromptTemplate construira deux messages lorsqu'il sera appelé. Le premier est un message système, qui n'a aucune variable à formater. Le second est un HumanMessage, et sera formaté par la variable sujet transmise par l'utilisateur.


ChatPromptValue(messages=[SystemMessage(content='Tu es un Rose, une slameuse', additional_kwargs={}, response_metadata={}), HumanMessage(content='Ecrit un texte de slam sur le amour', additional_kwargs={}, response_metadata={})])

### MessagesPlaceholder

Ce prompt est chargé d'ajouter une liste de messages à un endroit particulier. Dans le modèle ChatPromptTemplate ci-dessus, nous avons vu comment nous pouvions formater deux messages, chacun étant une chaîne de caractères. Mais qu'en est-il si nous voulons que l'utilisateur transmette une liste de messages que nous placerons à un endroit particulier ? C'est ainsi que l'on utilise MessagesPlaceholder.

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate([
    ("system", "Tu es un Rose, une slameuse"),
    MessagesPlaceholder("Memoire")
])

prompt_template.invoke({"Memoire": [HumanMessage(content="hi!")]})

ChatPromptValue(messages=[SystemMessage(content='Tu es un Rose, une slameuse', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])